**Find early pepe whales:**

In [2]:
#
from api_endpoints.etherscan_api import *
from api_endpoints.covalent_api import *

from etherscan import Etherscan #externally installed package https://github.com/pcko1/etherscan-python

Step 1): Find the first $n$ addresses for an address (`pepe`)

In [3]:
first_n_pepe = get_first_n_addresses(pepe_address, 100,api_key)

In [4]:
first_n_pepe_addresses = [address for address in first_n_pepe]

Definition 1: A `whale` is an address that has transacted $x$ units of ETH in and out of their address, for a large enough $x$ at the present date.

Problem: we might want to compute this, conversely, for $x$ units with respect to a past date, e.g. the day a token was launched. This probably makes more sense.

In [4]:
import requests
import json

def threshold_eth_transacted(address, x,api_key):
    "Return `True` if address has >=x units in and out, else `False`"
    #TODO: Can we make this relative to a past date?

    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={api_key}"
    response = requests.get(url)
    transactions = json.loads(response.text)['result']
    
    total = 0
    total_out = 0
    for txn in transactions:
        value = int(txn['value']) / (10 ** 18)  # Convert from Wei to Ether
        if txn['to'] == address.lower():
            total += value
        elif txn['from'] == address.lower():
            total += value

        if total>=x:
            return True
    return False
    

In [5]:
def get_large_addresses(address_lst: list, x: float,api_key: str) -> list:
    """Returns a list of addresses that have a balance greater than x."""
    
    return [address for address in address_lst if threshold_eth_transacted(address,x,api_key)]
    # _address_sum = {address:sum(get_total_eth_transacted(address,api_key)) for address in address_lst}

    # #return a list of addresses that have a balance greater than x
    # return [address for address in _address_sum if _address_sum[address] > x]

if __name__ == "__main__":

    _pepe_addresses = [first_n_pepe_addresses[i] for i in range(200,300)] #just skip the first few as it may have stuff like the bybit hot wallet etc...
    large_addresses = get_large_addresses(_pepe_addresses, 100.0,api_key)
    print(len(large_addresses))


56


Comments: the code might be a bit slow (nearly >5 mins to run). This may just be a consequence of `requests` being slow. Anyway, we found 56 addresses, between address 200 and address 300, that had more than 100.0 ETH transacted in and out. Consider the first one:

In [6]:
get_total_eth_transacted(large_addresses[0],api_key)

(1524.7574231098115, 7912.2039411033)

So ~ 1520.0+7900.0 ETH transacted, which is a lot (and certainly larger than 100.0 so it passes that sanity check).

In fact, this address could be considered a `whale.`

**Now we try using some functionality from the `covalent` API. NOTE: Also uses `get_creation_date` from `pepe_1`: #TODO: rename this file!**

One possibility: 
    - get ~ the date a token was launched (can just do this manually to start with).
    - compute the portfolio value (as above) for this date for several early addresses.
    - Then we need a method to convert the portfolio value to ETH or USD; e.g. what does "AllCoinsYieldCapital balance: 50269997031584481646646431" mean? Perhaps can use a dextools API or something. This shouldn't be too hard. (One problem though: we need to get the value of these tokens on the given date).

So basically: this implies a new definition of `whale`:

Definition 2: A  (early?) `whale` is a portfolio value across a threshold ~ the date a token was bought. e.g. if an address is the 13th one to purchase `pepe` (so an early buyer) and on the date they first did, their total wallet value was $> x$ USD for some large enough $x$ then we call them an `early whale`. Note that this is different to definition 1, where we just computed the total amount of ETH in an out of an address at the present time.


Definition 3: An early whale is: A wallet that is both i) early to buy the coin; and ii) is presently a top holder. Note that this definition is not perfect: it doesn't logically mean they got rich from buying the coin!

Definition 4: 

In [4]:
chainName='eth-mainnet'
network='mainnet'
cov_api = Covalent_Api()


In [5]:
#Step 1): get pepe creation date:
#First get pepe creation date (of contract, not commencement of trading!)

pepe_creation_date = get_creation_date(api_key, pepe_address, network=network)
pepe_creation_date = pepe_creation_date.split(' ')[0]
print(pepe_creation_date)

https://api.etherscan.io/api?
2023-04-14


In [7]:
#New step 2): get portfolios of each address at pepe creation date

#First, note that `get_holders_portfolios` requires list of dictionaries, which we will then update with the portfolios
first_n_pepe_addresses_lstofdict = [{'address':address} for address in first_n_pepe_addresses]

first_n_pepe_addresses_lstofdict=cov_api.get_holders_portfolios(n_lst=first_n_pepe_addresses_lstofdict,chainName=chainName,quote_currency="USD",date=pepe_creation_date,log_output=False)




TypeError: 'NoneType' object is not iterable

In [8]:
sum([])

0

In [8]:
first_n_pepe_addresses[0]

'0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a'

In [40]:
#Step 2): Get portfolios of the first 100 pepe addresses at pepe_creation_date

assert len(first_n_pepe_addresses) == 100

cov_api = Covalent_Api()

large_addresses_covalent = []
for address in first_n_pepe_addresses:
    items = cov_api.get_historical_balances(chainName='eth-mainnet',walletAddress=address, quote_currency="USD",date=pepe_creation_date)
    
    print(len(items))
    print(items)

    input()
    #portfolio_sum = sum([holding['quote'] for holding in items])

    #large_addresses_covalent.append({'address':address,'items':items})




#These are not 'large' addresses yet - we are just getting the portfolio value for each address first, at `date`.
#Then we will check if the portfolio is large enough to be considered a whale.
assert len(large_addresses_covalent) == 100

#_address = '0x6b75d8af000000e20b7a7ddf000ba900b4009a80' #the address that gave the "error"

1
[{'contract_decimals': 18, 'contract_name': 'Ether', 'contract_ticker_symbol': 'ETH', 'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'supports_erc': None, 'logo_url': 'https://www.datocms-assets.com/86369/1669619533-ethereum.png', 'block_height': 17041765, 'last_transferred_block_height': 0, 'last_transferred_at': None, 'native_token': True, 'type': 'dust', 'is_spam': False, 'balance': '0', 'quote_rate': 2012.332, 'quote': 0.0, 'pretty_quote': '$0.00', 'nft_data': None}]
1
[{'contract_decimals': 18, 'contract_name': 'Ether', 'contract_ticker_symbol': 'ETH', 'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'supports_erc': None, 'logo_url': 'https://www.datocms-assets.com/86369/1669619533-ethereum.png', 'block_height': 17041765, 'last_transferred_block_height': 0, 'last_transferred_at': None, 'native_token': True, 'type': 'dust', 'is_spam': False, 'balance': '0', 'quote_rate': 2012.332, 'quote': 0.0, 'pretty_quote': '$0.00', 'nft_data': None}]
1
[{'c

KeyboardInterrupt: Interrupted by user

In [57]:
len(large_addresses_covalent)

0

In [39]:
items[0]

{'contract_decimals': 18,
 'contract_name': 'Ether',
 'contract_ticker_symbol': 'ETH',
 'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee',
 'supports_erc': ['erc20'],
 'logo_url': 'https://www.datocms-assets.com/86369/1669619533-ethereum.png',
 'update_at': '2023-08-29T02:38:29.750967524Z',
 'quote_currency': 'USD',
 'prices': [{'contract_metadata': {'contract_decimals': 18,
    'contract_name': 'Ether',
    'contract_ticker_symbol': 'ETH',
    'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee',
    'supports_erc': None,
    'logo_url': 'https://www.datocms-assets.com/86369/1669619533-ethereum.png'},
   'date': '2023-04-14',
   'price': 2075.1746,
   'pretty_price': '$2,075.17'}],
 'items': [{'contract_metadata': {'contract_decimals': 18,
    'contract_name': 'Ether',
    'contract_ticker_symbol': 'ETH',
    'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee',
    'supports_erc': None,
    'logo_url': 'https://www.datocms-assets.com/86369/16696

Comment: this code *might* be a bit slow. #TODO: look into possibily speeding it up.

In [35]:
#Step 3: Define a mapping from an address / portfolio to a boolean (True if large, False if not large) fo given definition of large

for 

1

In [21]:
# #Step 3: update porfolios with USD value of each token at pepe_creation_date

# #Next map a portfolio to the dollar values:

# #First just do for a single guy

# address = large_addresses_covalent[0]['items']

# for portfolio in large_addresses_covalent:

#     for holding in portfolio['items']:

#         address = holding['contract_address']

#         print(f"The holding is: {holding} the address is: {address}\n")
    
#         items = cov_api.get_prices(chainName='eth-mainnet',address=address,quote_currency="USD",dates=[pepe_creation_date])
#         cov_api.print_prices(items)

#         price = items[0]['items'][0]['price'] #this is the price of the token at pepe_creation_date
#         value = price * float(holding['balance']) #this is the value of the holding in the porfolio at pepe_creation_date (i.e. price*balance)

#         print(f"The price is: {price} the balance is: {holding['balance']} the value is: {value}\n")

#         holding['value'] = value #the USD value of the holding at pepe_creation_date

#         print(f"After upading the `holding` dict, it is now: {holding}\n")

#         break
#     break



# # items = cov_api.get_prices(chainName='eth-mainnet',address=hpbitcoin,quote_currency="USD",dates=["2023-08-25","2023-08-26"])
# # print(f"Printing out prices of: {items[0]['contract_name']}")
# # cov_api.print_prices(items)

The holding is: {'contract_decimals': 18, 'contract_name': 'Ether', 'contract_ticker_symbol': 'ETH', 'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'supports_erc': None, 'logo_url': 'https://www.datocms-assets.com/86369/1669619533-ethereum.png', 'block_height': 17041765, 'last_transferred_block_height': 0, 'last_transferred_at': None, 'native_token': True, 'type': 'dust', 'is_spam': False, 'balance': '0', 'quote_rate': 2012.332, 'quote': 0.0, 'pretty_quote': '$0.00', 'nft_data': None} the address is: 0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

Printing out prices of: Ether
On 2023-04-14, the price was 2075.1746, and the `pretty_price` was $2,075.17
The price is: 2075.1746 the balance is: 0 the value is: 0.0

After upading the `holding` dict, it is now: {'contract_decimals': 18, 'contract_name': 'Ether', 'contract_ticker_symbol': 'ETH', 'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'supports_erc': None, 'logo_url': 'https://www.datocms-assets.com/8

Idea: compute `intersect_dict` from several "early whales" (according to whichever definition we use) -> then look at what they are holding now.